<a href="https://colab.research.google.com/github/dmdiegoar/Quant-code-t0/blob/main/caminos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gymnasium stable-baselines3 numpy matplotlib

import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO
import matplotlib.pyplot as plt

# Simular 1200 carreteras (reemplazar con tus datos reales)
roads = [[0] + [np.clip(np.random.uniform(-0.5, 0.5) + y, -1, 1) for y in [0]*31] for _ in range(1200)]
# Ejemplo de cómo cargar tus carreteras desde un CSV:
# import pandas as pd
# df = pd.read_csv("carreteras.csv")  # 1200 filas, 32 columnas
# roads = [row.tolist() for _, row in df.iterrows()]

class RoadFollowingEnv(gym.Env):
    def __init__(self, roads):
        super(RoadFollowingEnv, self).__init__()
        self.action_space = spaces.Discrete(3)  # 0: mantener, 1: izquierda, 2: derecha
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)  # x, y, ángulo, y_traza, delta_y1, delta_y2
        self.roads = roads
        self.current_road_idx = 0
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.velocity = 1.0
        self.step_count = 0
        self.max_steps = 1000
        self.current_road = self.roads[0]

    def reset(self):
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.step_count = 0
        self.current_road_idx = (self.current_road_idx + 1) % len(self.roads)  # Siguiente carretera
        self.current_road = self.roads[self.current_road_idx]
        return self._get_observation()

    def _get_observation(self):
        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        delta_y1 = self.current_road[min(idx + 1, len(self.current_road) - 1)] - self.current_road[idx]
        delta_y2 = self.current_road[min(idx + 2, len(self.current_road) - 1)] - self.current_road[min(idx + 1, len(self.current_road) - 1)]
        return np.array([self.position[0], self.position[1], self.angle, y_traza, delta_y1, delta_y2], dtype=np.float32)

    def step(self, action):
        self.step_count += 1
        if action == 1:  # Izquierda
            self.angle -= 0.1
        elif action == 2:  # Derecha
            self.angle += 0.1
        self.position[0] += self.velocity * np.cos(self.angle) * 0.1
        self.position[1] += self.velocity * np.sin(self.angle) * 0.1

        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        distance_to_track = abs(self.position[1] - y_traza)
        reward = 1.0 - distance_to_track
        if distance_to_track > 0.5:
            reward = -10.0
            done = True
        else:
            done = self.step_count >= self.max_steps or x >= len(self.current_road) - 1
        return self._get_observation(), reward, done, {}

    def render(self):
        plt.clf()
        x = np.arange(0, len(self.current_road), 1)
        y = self.current_road
        plt.plot(x, y, 'b-', label='Traza ideal')
        plt.plot(self.position[0], self.position[1], 'ro', label='Vehículo')
        plt.legend()
        plt.title(f"Carretera {self.current_road_idx}")
        plt.show()

# Entrenar el agente
env = RoadFollowingEnv(roads)
model = PPO("MlpPolicy", env, verbose=1)
episode_rewards = []
for episode in range(1200):  # Entrenar en cada carretera
    obs = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        steps += 1
        if steps % 100 == 0:
            env.render()
    avg_reward = total_reward / steps
    episode_rewards.append(avg_reward)
    print(f"Episodio {episode + 1}, Carretera {env.current_road_idx}, Recompensa promedio: {avg_reward:.2f}")
    if avg_reward > 0.8:  # Criterio de aprendizaje
        print(f"Carretera {env.current_road_idx} aprendida!")
    model.learn(total_timesteps=1000)  # Entrenar 1000 pasos por episodio

# Guardar el modelo
model.save("ppo_road_follower")

# Probar en una carretera nueva
test_road = [0, 0.3, -0.2, -0.5, 0.1, 0.7, -0.2, 0.4] + [0]*24  # Nueva carretera con 32 puntos
env.current_road = test_road
obs = env.reset()
positions = [(obs[0], obs[1])]
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    positions.append((obs[0], obs[1]))
    env.render()
    if done:
        break

# Visualizar resultados
plt.plot(range(32), test_road, 'b-', label='Traza ideal')
plt.plot([p[0] for p in positions], [p[1] for p in positions], 'r-', label='Vehículo')
plt.legend()
plt.title("Prueba en carretera nueva")
plt.show()

# Guardar modelo para descargar
from google.colab import files
files.download("ppo_road_follower.zip")

Caminos

In [ ]:
import pandas as pd
df = pd.read_csv("carreteras.csv")  # 1200 filas, 32 columnas
roads = [row.tolist() for _, row in df.iterrows()]

Giros

In [ ]:
!pip install gymnasium stable-baselines3 numpy matplotlib

import gym
import numpy as np
from gym import spaces
from stable_baselines3 import PPO
import matplotlib.pyplot as plt

# Simular 1200 carreteras (reemplazar con tus datos reales)
roads = [[0] + [np.clip(np.random.uniform(-0.5, 0.5) + y, -1, 1) for y in [0]*31] for _ in range(1200)]
# Ejemplo de cómo cargar tus carreteras desde un CSV:
# import pandas as pd
# df = pd.read_csv("carreteras.csv")  # 1200 filas, 32 columnas
# roads = [row.tolist() for _, row in df.iterrows()]

class RoadFollowingEnv(gym.Env):
    def __init__(self, roads):
        super(RoadFollowingEnv, self).__init__()
        self.action_space = spaces.Discrete(3)  # 0: mantener, 1: izquierda, 2: derecha
        self.observation_space = spaces.Box(low=-1, high=1, shape=(3,), dtype=np.float32)  # y_vehículo - y_traza, signo(delta_y1), signo(delta_y2)
        self.roads = roads
        self.current_road_idx = 0
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.velocity = 1.0
        self.step_count = 0
        self.max_steps = 1000
        self.current_road = self.roads[0]

    def reset(self):
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.step_count = 0
        self.current_road_idx = (self.current_road_idx + 1) % len(self.roads)  # Siguiente carretera
        self.current_road = self.roads[self.current_road_idx]
        return self._get_observation()

    def _get_observation(self):
        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        delta_y1 = self.current_road[min(idx + 1, len(self.current_road) - 1)] - self.current_road[idx]
        delta_y2 = self.current_road[min(idx + 2, len(self.current_road) - 1)] - self.current_road[min(idx + 1, len(self.current_road) - 1)]
        sign_delta_y1 = np.sign(delta_y1)
        sign_delta_y2 = np.sign(delta_y2)
        return np.array([self.position[1] - y_traza, sign_delta_y1, sign_delta_y2], dtype=np.float32)

    def step(self, action):
        self.step_count += 1
        if action == 1:  # Izquierda
            self.angle -= 0.1
        elif action == 2:  # Derecha
            self.angle += 0.1
        self.position[0] += self.velocity * np.cos(self.angle) * 0.1
        self.position[1] += self.velocity * np.sin(self.angle) * 0.1

        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        distance_to_track = abs(self.position[1] - y_traza)
        reward = 1.0 - distance_to_track
        if action != 0:  # Penalizar giros innecesarios
            delta_y1 = self.current_road[min(idx + 1, len(self.current_road) - 1)] - self.current_road[idx]
            delta_y2 = self.current_road[min(idx + 2, len(self.current_road) - 1)] - self.current_road[min(idx + 1, len(self.current_road) - 1)]
            if np.sign(delta_y1) == np.sign(delta_y2):  # No hay cambio de sentido
                reward -= 0.1
        if distance_to_track > 0.5:
            reward = -10.0
            done = True
        else:
            done = self.step_count >= self.max_steps or x >= len(self.current_road) - 1
        return self._get_observation(), reward, done, {}

    def render(self):
        plt.clf()
        x = np.arange(0, len(self.current_road), 1)
        y = self.current_road
        plt.plot(x, y, 'b-', label='Traza ideal')
        plt.plot(self.position[0], self.position[1], 'ro', label='Vehículo')
        plt.legend()
        plt.title(f"Carretera {self.current_road_idx}")
        plt.show()

# Entrenar el agente
env = RoadFollowingEnv(roads)
model = PPO("MlpPolicy", env, verbose=1)
episode_rewards = []
for episode in range(1200):  # Entrenar en cada carretera
    obs = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, done, _ = env.step(action)
        total_reward += reward
        steps += 1
        if steps % 100 == 0:
            env.render()
    avg_reward = total_reward / steps
    episode_rewards.append(avg_reward)
    print(f"Episodio {episode + 1}, Carretera {env.current_road_idx}, Recompensa promedio: {avg_reward:.2f}")
    if avg_reward > 0.8:  # Criterio de aprendizaje
        print(f"Carretera {env.current_road_idx} aprendida!")
    model.learn(total_timesteps=1000)  # Entrenar 1000 pasos por episodio

# Guardar el modelo
model.save("ppo_road_follower_change")

# Probar en una carretera nueva
test_road = [0, 0.3, -0.2, -0.5, 0.1, 0.7, -0.2, 0.4] + [0]*24  # Nueva carretera con 32 puntos
env.current_road = test_road
obs = env.reset()
positions = [(obs[0], obs[1])]
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    positions.append((obs[0], obs[1]))
    env.render()
    if done:
        break

# Visualizar resultados
plt.plot(range(32), test_road, 'b-', label='Traza ideal')
plt.plot([p[0] for p in positions], [p[1] for p in positions], 'r-', label='Vehículo')
plt.legend()
plt.title("Prueba en carretera nueva")
plt.show()

# Guardar modelo para descargar
from google.colab import files
files.download("ppo_road_follower_change.zip")

Probar modelos guardado

In [ ]:
from stable_baselines3 import PPO
model = PPO.load("ppo_road_follower_angle")  # O ppo_road_follower_change
env = RoadFollowingEnv(roads)  # Cargar con cualquier lista de carreteras
test_road = [0, 0.3, -0.2, -0.5, 0.1, 0.7, -0.2, 0.4] + [0]*24
env.current_road = test_road
obs = env.reset()
positions = [(obs[0], obs[1])]
for _ in range(1000):
    action, _ = model.predict(obs)
    obs, reward, done, _ = env.step(action)
    positions.append((obs[0], obs[1]))
    env.render()
    if done:
        break
plt.plot(range(32), test_road, 'b-', label='Traza ideal')
plt.plot([p[0] for p in positions], [p[1] for p in positions], 'r-', label='Vehículo')
plt.legend()
plt.title("Prueba en carretera nueva")
plt.show()

Verla

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Pega aquí tu carretera (32 puntos)
road = [0, 0.3, -0.2, -0.5, 0.1, 0.7, -0.2, 0.4, 0.0, 0.2, -0.1, 0.3, -0.3, 0.5, -0.4, 0.1, 0.0, 0.2, -0.2, 0.4, -0.5, 0.0, 0.3, -0.1, 0.2, -0.4, 0.5, -0.3, 0.1, 0.0, -0.2, 0.3]  # Ejemplo, reemplaza con tus 32 puntos

# Verificar que la carretera tenga 32 puntos
if len(road) != 32:
    print("Error: La carretera debe tener exactamente 32 puntos")
else:
    # Graficar
    x = np.arange(0, 32, 1)  # x = 0, 1, ..., 31
    y = road
    plt.figure(figsize=(8, 4))
    plt.plot(x, y, 'b-', marker='o', markersize=3, label='Carretera')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.title('Carretera con 32 puntos')
    plt.grid(True)
    plt.legend()
    plt.show()

Masima

In [ ]:
# Instalar dependencias
!pip uninstall -y gym gymnasium shimmy stable-baselines3 numpy
!pip install gymnasium==0.29.1 shimmy[gym-v26]>=0.2.0 stable-baselines3==2.0.0 numpy==1.26.4

import gymnasium as gym
import numpy as np
from gymnasium import spaces
from stable_baselines3 import PPO
import matplotlib.pyplot as plt
import pandas as pd
import time

# Simular 1200 carreteras (reemplazar con tus datos)
roads = [[0] + [np.clip(np.random.uniform(-1, 1) + y, -1, 1) for y in [0]*31] for _ in range(1200)]
# Para cargar desde CSV:
# import pandas as pd
# df = pd.read_csv("carreteras.csv")
# roads = [row.tolist() for _, row in df.iterrows()]

class RoadFollowingEnv(gym.Env):
    def __init__(self, roads=None, single_road=None):
        super(RoadFollowingEnv, self).__init__()
        self.action_space = spaces.Box(low=-0.3, high=0.3, shape=(1,), dtype=np.float32)
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(6,), dtype=np.float32)
        self.roads = roads
        self.single_road = single_road
        self.current_road_idx = -1
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.velocity = 1.0
        self.step_count = 0
        self.max_steps = 1000
        self.current_road = single_road if single_road is not None else None
        self.trajectory = []

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.position = np.array([0.0, 0.0])
        self.angle = 0.0
        self.step_count = 0
        self.trajectory = [(self.position[0], self.position[1])]
        if self.roads is not None:
            self.current_road_idx = (self.current_road_idx + 1) % len(self.roads)
            self.current_road = self.roads[self.current_road_idx]
        return self._get_observation(), {}

    def _get_observation(self):
        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        delta_y1 = 0 if idx == len(self.current_road) - 1 else self.current_road[idx + 1] - self.current_road[idx]
        delta_y2 = 0 if idx >= len(self.current_road) - 2 else self.current_road[idx + 2] - self.current_road[idx + 1]
        return np.array([self.position[0], self.position[1], self.angle, y_traza, delta_y1, delta_y2], dtype=np.float32)

    def step(self, action):
        self.step_count += 1
        self.angle += action[0]
        self.position[0] += self.velocity * np.cos(self.angle) * 0.1
        self.position[1] += self.velocity * np.sin(self.angle) * 0.1
        self.trajectory.append((self.position[0], self.position[1]))
        x = self.position[0]
        idx = int(x)
        if idx >= len(self.current_road) - 1:
            idx = len(self.current_road) - 2
        frac = x - idx
        y_traza = self.current_road[idx] + frac * (self.current_road[idx + 1] - self.current_road[idx])
        distance_to_track = abs(self.position[1] - y_traza)
        reward = 1.0 - distance_to_track
        terminated = distance_to_track > 0.5
        truncated = self.step_count >= self.max_steps or x >= len(self.current_road) - 1
        if terminated:
            reward = -10.0
        return self._get_observation(), reward, terminated, truncated, {}

    def render(self, episode, avg_reward, steps):
        plt.clf()
        x = np.arange(0, len(self.current_road), 1)
        y = self.current_road
        plt.plot(x, y, 'b-', marker='o', markersize=3, label='Carretera')
        traj_x = [p[0] for p in self.trajectory]
        traj_y = [p[1] for p in self.trajectory]
        plt.plot(traj_x, traj_y, 'r-', label='Trayectoria del vehículo')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title(f'Carretera {self.current_road_idx + 1} - Aprendida!')
        # Añadir cuadro con datos
        textstr = f'Episodio: {episode}\nCarretera: {self.current_road_idx + 1}\nRecompensa: {avg_reward:.2f}\nPasos: {steps}'
        plt.text(0.02, 0.98, textstr, transform=plt.gca().transAxes, fontsize=10,
                 verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
        plt.grid(True)
        plt.legend()
        plt.show()

# Entrenar el agente
env = RoadFollowingEnv(roads=roads)
model = PPO("MlpPolicy", env, verbose=0)
episode_rewards = []
log_data = []

for episode in range(1200):
    obs, _ = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action, _ = model.predict(obs)
        obs, reward, terminated, truncated, _ = env.step(action)
        total_reward += reward
        steps += 1
        done = terminated or truncated
    avg_reward = total_reward / steps
    episode_rewards.append(avg_reward)
    log_data.append({"Episode": episode + 1, "Road": env.current_road_idx + 1, "Avg_Reward": avg_reward})
    # Mostrar mensaje y gráfica solo si aprendió (recompensa > 0.8)
    if avg_reward > 0.8:
        print(f"Episodio {episode + 1}, Carretera {env.current_road_idx + 1}, Recompensa promedio: {avg_reward:.2f} - Aprendida!")
        env.render(episode + 1, avg_reward, steps)

# Guardar logs y modelo
pd.DataFrame(log_data).to_csv("training_log.csv", index=False)
model.save("ppo_road_follower_continuous")
print("Logs guardados en training_log.csv")
print("Modelo guardado como ppo_road_follower_continuous.zip")

# Visualizar conducción en carretera nueva
test_road = [0, 0.3, -0.2, -0.5, 0.1, 0.7, -0.2, 0.4, 0.0, 0.2, -0.1, 0.3, -0.3, 0.5, -0.4, 0.1,
             0.0, 0.2, -0.2, 0.4, -0.5, 0.0, 0.3, -0.1, 0.2, -0.4, 0.5, -0.3, 0.1, 0.0, -0.2, 0.3]
env = RoadFollowingEnv(single_road=test_road)
obs, _ = env.reset()
positions = [(obs[0], obs[1])]
for i in range(1000):
    action, _ = model.predict(obs)
    obs, reward, terminated, truncated, _ = env.step(action)
    positions.append((obs[0], obs[1]))
    if i % 10 == 0:  # Animación cada 10 pasos
        plt.clf()
        plt.plot(range(len(test_road)), test_road, 'b-', marker='o', markersize=3, label='Carretera')
        plt.plot([p[0] for p in env.trajectory], [p[1] for p in env.trajectory], 'r-', label='Trayectoria del vehículo')
        plt.xlabel('x')
        plt.ylabel('y')
        plt.title('Conducción en carretera nueva')
        plt.grid(True)
        plt.legend()
        plt.show()
        time.sleep(0.1)
    if terminated or truncated:
        break

# Gráfica final
plt.plot(range(len(test_road)), test_road, 'b-', marker='o', markersize=3, label='Carretera')
plt.plot([p[0] for p in positions], [p[1] for p in positions], 'r-', label='Trayectoria del vehículo')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Trayectoria completa en carretera nueva')
plt.grid(True)
plt.legend()
plt.show()

# Descargar resultados
from google.colab import files
files.download("ppo_road_follower_continuous.zip")
files.download("training_log.csv")